# 🎯 Enriquecimento por CINE — Merge restrito por Código do Curso

Gerado em **2025-11-07 14:00:40**.

Este notebook:
- padroniza nomes (`snake_case`);
- cria chaves **numéricas** de `codigo_do_curso` em ambas as planilhas;
- faz um **LEFT merge** importando **apenas**: codigo_area_geral_cine, area_geral_cine, codigo_area_detalhada_cine, area_detalhada_cine, codigo_area_especifica_cine, area_especifica_cine, rotulo_cine, avaliacao_oficial;
- adiciona **essas colunas ao final** da primeira planilha;
- quando os códigos não forem numéricos ou não coincidirem, os **novos campos ficam vazios**.


In [1]:
import pandas as pd
import numpy as np
import re, unicodedata
from pathlib import Path

In [2]:
# ========= 1) Normalização no formato =========
def normalize_upper(s: str) -> str:
    s = unicodedata.normalize("NFKD", str(s)).encode("ASCII", "ignore").decode("ASCII")
    s = s.strip()                       # remove espaços antes/depois
    s = re.sub(r"\s+", "_", s)          # espaços internos → underscore
    s = s.upper()                       # caixa alta
    s = re.sub(r"[^A-Z0-9_]", "", s)    # remove tudo que não for A-Z, 0-9 ou _
    s = re.sub(r"_+", "_", s)           # colapsa múltiplos _
    return s

# Detecta a coluna de "código do curso" após normalizar para UPPER_UNDERSCORE
def find_codigo_curso_col(cols_upper):
    preferred = [
        "CODIGO_DO_CURSO", "CODIGO_CURSO", "COD_CURSO",
        "CODIGO_DO_CURSO_REGULACAO", "CODIGO_CURSO_REGULACAO",
        "COD_CURSO_REGULACAO"
    ]
    for p in preferred:
        if p in cols_upper:
            return p
    # Fallback: qualquer coisa que tenha "COD" e "CURSO" nessa ordem
    for c in cols_upper:
        if re.search(r"COD.*CURSO", c):
            return c
    return None

In [3]:
# ========= 2) Entradas =========
data = "recredenciamento_nov.xlsx"   # planilha 1 (vai receber as colunas novas no fim)
curso = "data (1).xlsx"      # planilha 2 (fonte dos campos CINE/avaliação)
df = pd.read_excel(data, dtype=str)
mg = pd.read_excel(curso, dtype=str)

C:\Users\MEC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [4]:
# ========= 3) Normaliza nomes de colunas no formato solicitado =========
df.columns = [normalize_upper(c) for c in df.columns]
mg.columns = [normalize_upper(c) for c in mg.columns]

In [5]:
# ========= 4) Localiza a coluna de código em cada base =========
col_df = find_codigo_curso_col(set(df.columns))
col_mg = find_codigo_curso_col(set(mg.columns))

if not col_df:
    print("Colunas df:", df.columns.tolist())
    raise KeyError("Não encontrei a coluna de CÓDIGO DO CURSO em 'autorizações.xlsx' após normalização.")
if not col_mg:
    print("Colunas mg:", mg.columns.tolist())
    raise KeyError("Não encontrei a coluna de CÓDIGO DO CURSO em 'data (1).xlsx' após normalização.")

In [6]:
# ========= 5) Cria chave numérica (só casa quando for número igual) =========
df["_CODCURSO_NUM"] = pd.to_numeric(df[col_df], errors="coerce")
mg["_CODCURSO_NUM"] = pd.to_numeric(mg[col_mg], errors="coerce")

# opcional: se houver duplicados no mg pelo código, manter o 1º não nulo por coluna
if mg["_CODCURSO_NUM"].duplicated().any():
    # prioriza registros mais completos
    mg = (mg.sort_values(by=[ "_CODCURSO_NUM" ])
            .groupby("_CODCURSO_NUM", as_index=False)
            .first())

In [7]:
# ========= 6) Seleciona apenas os campos a importar da planilha 2 =========
targets_human = [
    "Código Área Geral (CINE)",
    "Área Geral (CINE)",
    "Código Área Detalhada (CINE)",
    "Área Detalhada (CINE)",
    "Código Área Específica (CINE)",
    "Área Específica (CINE)",
    "Rótulo (CINE)",
    "Avaliação Oficial",
]
targets = [normalize_upper(c) for c in targets_human]

# garante existência (se faltar, fica vazio)
for c in targets:
    if c not in mg.columns:
        mg[c] = np.nan

mg_sel = mg[["_CODCURSO_NUM"] + targets].copy()

In [8]:
# ========= 7) LEFT MERGE: somente números iguais preenchem; demais ficam NaN =========
merged = df.merge(mg_sel, on="_CODCURSO_NUM", how="left")

# ========= 8) Ordena: novas colunas no FINAL e remove helper =========
new_cols = [c for c in targets if c in merged.columns]
base_cols = [c for c in merged.columns if c not in new_cols and c != "_CODCURSO_NUM"]
final = merged[base_cols + new_cols].copy()

In [9]:
# ================================================
# 9) REMOVER LINHAS COM NÚMEROS DE PROCESSO DUPLICADOS
# ================================================

COL_PROCESSO = "NO_DO_PROCESSO"   # coluna que contém o número do processo

if COL_PROCESSO not in df.columns:
    raise KeyError(f"A coluna '{COL_PROCESSO}' não existe no DataFrame. Verifique o nome após a normalização.")

# Contagem antes
total_antes = df.shape[0]

# Quantidade de duplicados
qtd_duplicados = df[COL_PROCESSO].duplicated(keep='first').sum()
print(f"🔍 Processos duplicados encontrados: {qtd_duplicados}")

# Remover duplicados mantendo a PRIMEIRA ocorrência
df = df.drop_duplicates(subset=[COL_PROCESSO], keep='first').reset_index(drop=True)

total_depois = df.shape[0]
removidas = total_antes - total_depois

print(f"✅ Linhas removidas por duplicidade: {removidas}")
print(f"📊 Total de linhas finais após deduplicação: {total_depois}")


🔍 Processos duplicados encontrados: 1531
✅ Linhas removidas por duplicidade: 1531
📊 Total de linhas finais após deduplicação: 2131


In [10]:
# ========= 10) Exporta =========
Path("artifacts").mkdir(exist_ok=True)
out_csv = "artifacts/RECREDENCIAMENTO_CINE.csv" #trocar os nomes dos arquivos finais
out_xlsx = "artifacts/RECREDENCIAMENTO_CINE.xlsx"
final.to_csv(out_csv, index=False, encoding="utf-8-sig")
try:
    import openpyxl  # opcional
    final.to_excel(out_xlsx, index=False)
except Exception:
    pass

print("✅ Pronto!")
print("Salvei:", out_csv, "e (se disponível) também", out_xlsx)


✅ Pronto!
Salvei: artifacts/RECREDENCIAMENTO_CINE.csv e (se disponível) também artifacts/RECREDENCIAMENTO_CINE.xlsx
